In [3]:
import csv
import gzip
import io

import os
import sys
import glob
import datetime
import warnings
from functools import partial
from unicodedata import normalize

import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas_profiling import ProfileReport

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns



warnings.filterwarnings("ignore")

%matplotlib inline

color = (0.45, 0.57, 1)
sns.set(style="whitegrid")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('use_inf_as_na', True)

PROJECT_NAME = 'UFAL-DS'
#curdir = os.path.abspath(os.path.curdir).split('\\')
#project_index = curdir.index(PROJECT_NAME)
#os.chdir('/' + os.path.join(*curdir[:project_index + 1]))

In [4]:
directory:str = "data/raw/abono/*.csv"

In [5]:
cols_dict = {
    'Denominação do órgão de atuação' : 'agency', 
    'Val' : 'value', 'UF da Residência' : 'state',
    'Denominação unidade organizacional' : 'unit', 
    'Nível de Escolaridade' : 'educational_level', 
    'Quantidade de anos no Serviço público' : 'years',
    'Quantidade de meses no Serviço público' : 'months',
    'Descrição do cargo emprego' : 'role', 'CPF' : 'cpf',
    'UF da UPAG de vinculação' : 'upag_state', 'Nome' : 'name',
    'Situação servidor' : 'status', 'Cidade da residência' : 'city',
    'Ano/Mês inicial do abono de permanência' : 'allowance_start_date',
}

cat_cols = [
    'name', 'role', 'educational_level', 'agency', 
    'upag_state', 'unit', 'state', 'city', 'status',
]

num_cols = [
    'years', 'months'
]

In [11]:
from IPython import get_ipython
get_ipython().magic('reset -sf') 

In [12]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [9]:
def sanitize(df: DataFrame) -> DataFrame:
    df = df.rename(columns=cols_dict)
    
    df['value'] = df['value'].str.replace(',', '.').astype(float)
    df['allowance_start_date'] = pd.to_datetime(
        df['allowance_start_date'], format='%Y%m'
    )
    
    for col in cat_cols:
        df[col] = df[col].str.lower()
    
    return df


def read_data(directory:str = "C:/Users/wagne/Documents/Aninha/Projetos/data/raw/abono/*.csv") -> DataFrame:
    df = pd.concat(
        [
            pd.read_csv(file, encoding="ISO-8859-1", sep=';') 
            for file in glob.glob(directory)
        ],
        ignore_index=False
    )
    
    df = df.reset_index()
    new_columns = list(df.columns[1:]) + ["NaN"]
    cols = {
        old:new for old, new in zip(df.columns, new_columns)
    }
    
    df.rename(columns=cols, inplace=True)
    df.drop("NaN", axis=1, inplace=True)
    
    df = sanitize(df)
    
    return df

In [10]:
dataset = read_data()
dataset.head()

,name,cpf,role,educational_level,agency,upag_state,unit,state,city,status,years,months,allowance_start_date,value
0,cicero joaquim dos santos filho ...,***204574**,auditor fiscal federal agropecuario,ensino superior,"minist.da agricultura,pecuaria e abast.",pe,sup fed agric pec abast est pernambuco,pe,caruaru,ativo permanente,43,8,2003-12-01,2241.38
1,paulo elisio de faria ...,***402121**,agente administrativo,ensino superior,"minist.da agricultura,pecuaria e abast.",df,coord geral adm de pessoas,df,brasilia,ativo permanente,41,7,2010-01-01,558.32
2,maria vanilda dos santos ...,***184061**,agente administrativo,ensino medio,"minist.da agricultura,pecuaria e abast.",df,coord geral adm de pessoas,df,brasilia,ativo permanente,41,6,2009-04-01,550.73
3,sonia candida batista ...,***086171**,contador,ensino superior,"minist.da agricultura,pecuaria e abast.",df,coord geral adm de pessoas,df,brasilia,ativo em outro orgao,41,5,2009-09-01,1070.10
4,maria graci da conceicao maxima ...,***803611**,agente de portaria,ensino medio,"minist.da agricultura,pecuaria e abast.",df,coord geral adm de pessoas,df,brasilia,ativo em outro orgao,41,3,2003-12-01,543.75


In [13]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
dataset.shape

In [ ]:
dataset.drop_duplicates(subset="name",  keep="last")

In [ ]:
dataset.dtypes

In [ ]:
dataset.drop(dataset[dataset.allowance_start_date < '2003-12-16'].index)

In [ ]:
#dataset['role_gp'] = dataset['role'].str.strip()
dataset.loc[dataset.role.str.contains("medico"), "role_gp"] = "medico"
dataset.loc[dataset.role.str.contains("psicologo"), "role_gp"] = "psicologo"
dataset.loc[dataset.role.str.contains("odont"), "role_gp"] = "odontologo"
dataset.loc[dataset.role.str.contains("enfermei"), "role_gp"] = "enfermeiro"
dataset.loc[dataset.role.str.contains("tecnico"), "role_gp"] = "tecnico"
dataset.loc[dataset.role.str.contains("tec"), "role_gp"] = "tecnico"
dataset.loc[dataset.role.str.contains("professor"), "role_gp"] = "professor"
dataset.loc[dataset.role.str.contains("agente"), "role_gp"] = "agente"
dataset.loc[dataset.role.str.contains("ag "), "role_gp"] = "agente"
dataset.loc[dataset.role.str.contains("aux"), "role_gp"] = "auxiliar"
dataset.loc[dataset.role.str.contains("analista"), "role_gp"] = "analista"
dataset.loc[dataset.role.str.contains("assistente"), "role_gp"] = "assistente"
dataset.loc[dataset.role.str.contains("operador"), "role_gp"] = "operador"
dataset.loc[dataset.role.str.contains("policia"), "role_gp"] = "policia"
dataset.loc[dataset.role.str.contains("engenhe"), "role_gp"] = "engenheiro"
dataset.loc[dataset.role.str.contains("especialista"), "role_gp"] = "especialista"
dataset.loc[dataset.role.str.contains("servente"), "role_gp"] = "servente"
dataset.loc[dataset.role.str.contains("auditor"), "role_gp"] = "auditor"
dataset.loc[dataset.role.str.contains("administrador"), "role_gp"] = "administrador"
dataset.loc[dataset.role.str.contains("motorista"), "role_gp"] = "motorista"
dataset.loc[dataset.role.str.contains("datilografo"), "role_gp"] = "datilografo"
dataset.loc[dataset.role.str.contains("procurador"), "role_gp"] = "procurador"
dataset.loc[dataset.role.str.contains("advogado"), "role_gp"] = "advogado"
dataset.loc[dataset.role.str.contains("recepcionista"), "role_gp"] = "recepcionista"
dataset.loc[dataset.role.str.contains("pesq"), "role_gp"] = "pesquisador"
dataset.loc[dataset.role.str.contains("esp"), "role_gp"] = "especialista"

dataset.loc[dataset.role_gp.isna(), "role_gp"] = "outros" 

In [ ]:
dataset['agency_gp'] = dataset['agency'].str.strip()

dataset.loc[dataset.agency_gp.str.contains("instituto federal"), "agency_gp"] = "instituto"
dataset.loc[dataset.agency_gp.str.contains("instituto fed."), "agency_gp"] = "instituto_federal"
dataset.loc[dataset.agency_gp.str.contains("colegio pedro ii"), "agency_gp"] = "instituto_federal"
dataset.loc[dataset.agency_gp.str.contains("instituto"), "agency_gp"] = "instituto"
dataset.loc[dataset.agency_gp.str.contains("universidade federal"), "agency_gp"] = "universidade"
dataset.loc[dataset.agency_gp.str.contains("universidade fed."), "agency_gp"] = "universidade"
dataset.loc[dataset.agency_gp.str.contains("univ."), "agency_gp"] = "universidade"
dataset.loc[dataset.agency_gp.str.contains("uni.fed"), "agency_gp"] = "universidade"
dataset.loc[dataset.agency_gp.str.contains("fund. inst. brasil. geog. e estatistica"), "agency_gp"] = "ibge"
dataset.loc[dataset.agency_gp.str.contains("fundacao"), "agency_gp"] = "fundacao"
dataset.loc[dataset.agency_gp.str.contains("fund"), "agency_gp"] = "fundacao"
dataset.loc[dataset.agency_gp.str.contains("instituto nacional de seguro social"), "agency_gp"] = "inss"
dataset.loc[dataset.agency_gp.str.contains("agencia"), "agency_gp"] = "agencia"
dataset.loc[dataset.agency_gp.str.contains("minist"), "agency_gp"] = "ministerio"
dataset.loc[dataset.agency_gp.str.contains("superin"), "agency_gp"] = "superintendencia"
dataset.loc[dataset.agency_gp.str.contains("advocacia"), "agency_gp"] = "advocacia"

#dataset.loc[dataset.agency_gp.isna(), "agency_gp"] = "outros" 

In [ ]:
plt.figure(figsize=(15,8))

chart = sns.boxplot(
    x="agency_gp", y="value", 
    data=dataset, color=color, showfliers=False
)

_ = chart.set_xticklabels(chart.get_xticklabels(), rotation=90, horizontalalignment='right')

In [ ]:
dataset.loc[dataset.upag_state == "pi", "region"] = "nordeste"
dataset.loc[dataset.upag_state == "ma", "region"] = "nordeste"
dataset.loc[dataset.upag_state == "ce", "region"] = "nordeste"
dataset.loc[dataset.upag_state == "rn", "region"] = "nordeste"
dataset.loc[dataset.upag_state == "pb", "region"] = "nordeste"
dataset.loc[dataset.upag_state == "pe", "region"] = "nordeste"
dataset.loc[dataset.upag_state == "al", "region"] = "nordeste"
dataset.loc[dataset.upag_state == "se", "region"] = "nordeste"
dataset.loc[dataset.upag_state == "ba", "region"] = "nordeste"

dataset.loc[dataset.upag_state == "es", "region"] = "sudeste"
dataset.loc[dataset.upag_state == "rj", "region"] = "sudeste"
dataset.loc[dataset.upag_state == "sp", "region"] = "sudeste"
dataset.loc[dataset.upag_state == "mg", "region"] = "sudeste"

dataset.loc[dataset.upag_state == "ac", "region"] = "norte"
dataset.loc[dataset.upag_state == "am", "region"] = "norte"
dataset.loc[dataset.upag_state == "ro", "region"] = "norte"
dataset.loc[dataset.upag_state == "rr", "region"] = "norte"
dataset.loc[dataset.upag_state == "am", "region"] = "norte"
dataset.loc[dataset.upag_state == "pa", "region"] = "norte"
dataset.loc[dataset.upag_state == "to", "region"] = "norte"

dataset.loc[dataset.upag_state == "df", "region"] = "centro-oeste"
dataset.loc[dataset.upag_state == "go", "region"] = "centro-oeste"
dataset.loc[dataset.upag_state == "mt", "region"] = "centro-oeste"
dataset.loc[dataset.upag_state == "ms", "region"] = "centro-oeste"

dataset.loc[dataset.upag_state == "pr", "region"] = "sul"
dataset.loc[dataset.upag_state == "rs", "region"] = "sul"
dataset.loc[dataset.upag_state == "sc", "region"] = "sul"



In [ ]:
dataset.loc[
    (dataset.educational_level == '4a. serie do primeiro grau completa          ') |
    (dataset.educational_level == 'ensino fundamental incompleto                ') |
    (dataset.educational_level == 'primeiro grau incomp.-ate a 4a.serie incomp. '),
    "educational_level"
] = "1º grau incompleto"

dataset.loc[
    dataset.educational_level == 'segundo grau incompleto                      ',
    "educational_level"
] = "2º grau incompleto"

dataset.loc[
    dataset.educational_level == 'alfabetizado sem cursos regulares            ',
    "educational_level"
] = "alfabetizado"


dataset["educational_level"] = dataset["educational_level"].str.rstrip()

### Analisando o gênero 
A analise de gênero do beneficiário é realizada com base no Censo 2010.

In [ ]:
def load_name_data():
    fobj = io.TextIOWrapper(gzip.open("C:/Users/Aninha/Documents/Area de Trabalho/UFAL-DS/data/raw/censo/nomes.csv.gz"), encoding="utf-8")
    csv_reader = csv.DictReader(fobj)
    data = {
        row["first_name"]: row["classification"]
        for row in csv_reader
    }
    fobj.close()
    return data

name_data = load_name_data()

print(f"Dicionário de nomes do Censo 2010 contendo {len(name_data)} nomes.")

In [ ]:
def encode(name):
    ascii_name = normalize("NFKD", name).encode("ascii", errors="ignore").decode("ascii")
    return ascii_name.upper()

def classify_gender(name):
    encoded_name = encode(name)
    try:
        gender = name_data[encoded_name]
        return gender
    except:
        return "NaN"

print(classify_gender("Wagner"))

In [ ]:
dataset["gender"] = dataset.name.str.split(" ").str[0]
dataset["gender"] = dataset.gender.map(classify_gender)

In [ ]:
f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (0.25, 1)}, figsize=(16,8))
mean = dataset.value.mean()
median = dataset.value.median()

_ = sns.boxplot(dataset.value, ax=ax_box,  color=color, showfliers=True)
_ = ax_box.axvline(mean, color='r', linestyle='--')
_ = ax_box.axvline(median, color='g', linestyle='-')
_ = ax_box.set(xlabel='')

_ = sns.distplot(dataset.value, ax=ax_hist, color=color, kde=False)
_ = ax_hist.axvline(mean, color='r', linestyle='--')
_ = ax_hist.axvline(median, color='g', linestyle='-')
_ = ax_hist.xaxis.set_major_locator(ticker.MultipleLocator(5000))

_ = plt.legend({'Mean':mean,'Median':median}, fontsize='large')

### Criação do csv com os dados preprocessados

In [ ]:
dataset.to_csv('C:/Users/Aninha/Documents/Area de Trabalho/UFAL-DS/data/raw/preprocessing/abono.csv', sep=',')

In [ ]:
dataset = pd.read_csv('C:/Users/Aninha/Documents/Area de Trabalho/UFAL-DS/data/raw/preprocessing/abono.csv', sep=',')
dataset.head()

In [ ]:
## Tirando os outliers
![title](https://www.gradusct.com.br/wp-content/uploads/2018/07/Boxplot-04.png)

In [ ]:
values = dataset['value']

q1 = values.quantile(.25)
q3 = values.quantile(.75)
iiq = q3 - q1
minimum = q1 - 1.5*iiq
maximum = q3 + 1.5*iiq

selection = (values >= minimum) & (values <= maximum)
data_new = dataset[selection]

selection = (values <= minimum) | (values >= maximum)
data_outliers = dataset[selection]

## Análise dos dados (sem os Outiliers)

In [ ]:
profile = ProfileReport(data_new, title='Abono Permanencia',html={'style':{'full_width':True}})
profile.to_notebook_iframe()

In [ ]:
gp = data_new.groupby('region')
levels = data_new.region.value_counts()

fig = plt.figure(figsize=(14,30))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

total_plots = levels.shape[0]
i = 0
for level in levels.keys():
    ax = fig.add_subplot(total_plots, 2, i+1)
    ax.set_title(level)
    
    top = gp.get_group(level)["gender"].value_counts()[:3]
    _ = sns.barplot(x=top.keys(), y=top.values, color=color, ax=ax)
    
    i += 1

In [ ]:
plt.figure(figsize=(15,8))
ax = sns.distplot(data_new.value, bins=10)

In [ ]:
data_new["educational_level"].value_counts()

In [ ]:
f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (0.25, 1)}, figsize=(16,8))
mean = data_new.value.mean()
median = data_new.value.median()

_ = sns.boxplot(data_new.value, ax=ax_box,  color=color, showfliers=False)
_ = ax_box.axvline(mean, color='r', linestyle='--')
_ = ax_box.axvline(median, color='g', linestyle='-')
_ = ax_box.set(xlabel='')

_ = sns.distplot(data_new.value, ax=ax_hist, color=color, kde=False)
_ = ax_hist.axvline(mean, color='r', linestyle='--')
_ = ax_hist.axvline(median, color='g', linestyle='-')
_ = ax_hist.xaxis.set_major_locator(ticker.MultipleLocator(5000))

_ = plt.legend({'Mean':mean,'Median':median}, fontsize='large')

In [ ]:
data_new.boxplot(['value'], by = ['role_gp'], figsize = (15,8), rot=90, fontsize = 10)

In [ ]:
gp = data_new.groupby('role_gp')
levels = data_new.role_gp.value_counts()

fig = plt.figure(figsize=(14,60))
fig.subplots_adjust(hspace=0.8, wspace=0.4)

total_plots = levels.shape[0]
i = 0
for level in levels.keys():
    ax = fig.add_subplot(total_plots, 2, i+1)
    ax.set_title(level)
    
    top = gp.get_group(level)["agency_gp"].value_counts()[:3]
    _ = sns.barplot(x=top.keys(), y=top.values, color=color, ax=ax)
    
    i += 1

In [ ]:
gp = data_new.groupby('role_gp')
levels = data_new.role_gp.value_counts()

fig = plt.figure(figsize=(14,60))
fig.subplots_adjust(hspace=0.8, wspace=0.4)

total_plots = levels.shape[0]
i = 0
for level in levels.keys():
    ax = fig.add_subplot(total_plots, 2, i+1)
    ax.set_title(level)
    
    top = gp.get_group(level)["region"].value_counts()
    _ = sns.barplot(x=top.keys(), y=top.values, color=color, ax=ax)
    
    i += 1

In [ ]:
print("Mediana:", data_new.value.median())
print("Média", data_new.value.mean())
print("Percentil", data_new.value.quantile(0.75))
print("Desvio Padrão", data_new.value.std())

In [ ]:
states = data_new.groupby('state')
order = states.value.median().sort_values(ascending=True)
order

In [ ]:
plt.figure(figsize=(15,8))
states = data_new.groupby('state')
order = states.value.median().sort_values(ascending=True).keys().to_list()

chart = sns.boxplot(
    x="upag_state", y="value", 
    data=data_new, color=color, showfliers=False, order=order
)

_ = chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

In [ ]:
plt.figure(figsize=(15,8))
states = data_new.groupby('region')
order = states.value.median().sort_values(ascending=True).keys().to_list()

chart = sns.boxplot(
    x="region", y="value", 
    data=data_new, color=color, showfliers=False, order = order
)

_ = chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

In [ ]:
plt.figure(figsize=(20,8))
_ = sns.lineplot(x="allowance_start_date", y="value", data=data_new)

In [ ]:
# Prepare data
data_new['start_year'] = [d.year for d in data_new.allowance_start_date]
data_new['start_month'] = [d.strftime('%b') for d in data_new.allowance_start_date]
years = data_new['start_year'].unique()

fig, axes = plt.subplots(2, 1, figsize=(15,16), dpi= 80)
_ = sns.boxplot(x='start_year', y='value', data=data_new, ax=axes[0], showfliers=False)
_ = sns.boxplot(x='start_month', y='value', data=data_new.loc[~data_new.start_year.isin([2000, 2020]), :], showfliers=False)

# Set Title
axes[0].set_title('Year-wise Box Plot\n(The Trend)', fontsize=18); 
axes[1].set_title('Month-wise Box Plot\n(The Seasonality)', fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(15,8))

chart = sns.boxplot(
    x="educational_level", y="value", 
    data=data_new, color=color, showfliers=False
)

_ = chart.set_xticklabels(chart.get_xticklabels(), rotation=90, horizontalalignment='right')

In [ ]:
ncount = data_new.shape[0]

plt.figure(figsize=(14,10))

order = [
    'alfabetizado', '1º grau incompleto', 'ensino fundamental',
    '2º grau incompleto', 'ensino medio', 'superior incompleto',
    'ensino superior',  'mestrado', 'doutorado'
]

ax = sns.countplot(
    x="educational_level", data=data_new, 
    color=color, order=order
)
plt.xlabel('educational level')

ax2=ax.twinx()

ax2.yaxis.tick_left()
ax.yaxis.tick_right()

ax.yaxis.set_label_position('right')
ax2.yaxis.set_label_position('left')

ax2.set_ylabel('Frequency [%]')

for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y), 
            ha='center', va='bottom') 

ax.yaxis.set_major_locator(ticker.LinearLocator(11))

_ = ax.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

In [ ]:
data_new["educational_level"].value_counts()

## Análise dos Outliers

In [ ]:
plt.figure(figsize=(15,8))
ax = sns.distplot(data_outliers.value, bins=10)

In [ ]:
profile_outliers = ProfileReport(data_new, title='Abono Permanencia - Outliers',html={'style':{'full_width':True}})
profile_outliers.to_notebook_iframe()

In [ ]:
data_outliers.boxplot(['value'], by = ['agency_gp'], figsize = (15,8), rot=90, fontsize = 10)

In [ ]:
data_outliers.boxplot(['value'], by = ['role_gp'], figsize = (15,8), rot=90, fontsize = 10)

In [ ]:
gp = data_outliers.groupby('region')
levels = data_outliers.region.value_counts()

fig = plt.figure(figsize=(14,30))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

total_plots = levels.shape[0]
i = 0
for level in levels.keys():
    ax = fig.add_subplot(total_plots, 2, i+1)
    ax.set_title(level)
    
    top = gp.get_group(level)["gender"].value_counts()[:3]
    _ = sns.barplot(x=top.keys(), y=top.values, color=color, ax=ax)
    
    i += 1

In [ ]:
gp = data_outliers.groupby('role_gp')
levels = data_outliers.role_gp.value_counts()

fig = plt.figure(figsize=(14,60))
fig.subplots_adjust(hspace=0.8, wspace=0.4)

total_plots = levels.shape[0]
i = 0
for level in levels.keys():
    ax = fig.add_subplot(total_plots, 2, i+1)
    ax.set_title(level)
    
    top = gp.get_group(level)["region"].value_counts()
    _ = sns.barplot(x=top.keys(), y=top.values, color=color, ax=ax)
    
    i += 1

In [ ]:
dataset[dataset.gender == 'M'].educational_level.value_counts()

In [ ]:
plt.figure(figsize=(15,8))

chart = sns.countplot(x="educational_level", hue="gender", data=data_new)

_ = chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')